In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
ROTOR_WIRINGS = {
    'I': {'forward':'EKMFLGDQVZNTOWYHXUSPAIBRCJ'.lower(),
          'backward':'UWYGADFPVZBECKMTHXSLRINQOJ'.lower()},
    'II':{'forward':'AJDKSIRUXBLHWTMCQGZNPYFVOE'.lower(),
          'backward':'AJPCZWRLFBDKOTYUQGENHXMIVS'.lower()},
    'III':{'forward':'BDFHJLCPRTXVZNYEIWGAKMUSQO'.lower(),
           'backward':'TAGBPCSDQEUFVNZHYIXJWLRKOM'.lower()},
}


In [36]:
class Rotor:
    """
    key: int(0-26)
    rotor_type: "I", "II", "III"
    """
    def __init__(self, key, rotor_type="I"):
        self.alphabet = "abcdefghijklmnopqrstuvwxyz"
        self.key = key
        self.rotor_type = rotor_type
        self.wiring = ROTOR_WIRINGS[rotor_type]
    
    def step(self):
        self.key = (self.key+1) % 26
        
    def encode_letter(self, l, forward=True):
        if forward:
            index = (self.alphabet.index(l) + self.key) % 26
            l = self.wiring['forward'][index]
            index = (self.alphabet.index(l) - self.key) % 26
            return self.alphabet[index]
        else:
            index = (self.alphabet.index(l) + self.key) % 26
            l = self.wiring['backward'][index]
            index = (self.alphabet.index(l) - self.key) % 26
            return self.alphabet[index]

    def set_key(self, key):
        self.key = key

In [37]:
class Reflector:
    def __init__(self):
        # Reflector B of the Wehrmaht Engima.
        self.wiring = {'a':'y', 'b':'r', 'c':'u', 'd':'h', 'e':'q', 'f':'s', 'g':'l', 'h':'d',
                       'i':'p', 'j':'x', 'k':'n', 'l':'g', 'm':'o', 'n':'k', 'o':'m', 'p':'i',
                       'q':'e', 'r':'b', 's':'f', 't':'z', 'u':'c', 'v':'w', 'w':'v', 'x':'j',
                       'a':'a', 'z':'t'}
    def encrypt(self, l):
        return self.wiring[l]

In [51]:
class Plugboard:
    def __init__(self, plugs):
        # plugs = ["ab", "gd"...]
        self.plugs = {}
        ls = []
        for i in plugs:
            l1 = i[0]
            l2 = i[1]
            if l1 not in ls and l2 not in ls:
                self.plugs[l1] = l2
        self.invert_plugs = {v: k for k, v in self.plugs.items()}
        # {"a":"b", "g":"d",...}
    def encrypt(self, l, forward=True):
        if forward:
            try:
                return self.plugs[l]
            except:
                return l
            
        else:
            try:
                return self.invert_plugs[l]
            except:
                return l

In [49]:
class Enigma:
    def __init__(self, key=[0, 0, 0], plugboard=[], rotor_type=["I", "II", "III"]):
        self.key = key
        self.plugboard = Plugboard(plugboard)
        self.rotors = [Rotor(key[2 - i], rotor_type=rotor_type[i]) for i in range(3)]
        self.reflector = Reflector()

    def step_rotors(self):
        # Step the rightmost rotor every time
        self.rotors[0].step()
        
        # If rightmost rotor has completed a full rotation, step the middle rotor
        if self.rotors[0].key == 0:
            self.rotors[1].step()
        
        # If middle rotor has completed a full rotation, step the leftmost rotor
        if self.rotors[1].key == 0 and self.rotors[0].key == 0:
            self.rotors[2].step()

    def encrypt_letter(self, l):
        # Pass the letter through the plugboard
        l = self.plugboard.encrypt(l)

        # Forward pass through rotors
        for rotor in self.rotors:
            l = rotor.encode_letter(l, forward=True)

        # Pass through reflector
        l = self.reflector.encrypt(l)

        # Backward pass through rotors in reverse order
        for rotor in reversed(self.rotors):
            l = rotor.encode_letter(l, forward=False)

        # Pass the letter back through the plugboard
        l = self.plugboard.encrypt(l, forward=False)

        return l

    def encrypt(self, message):
        message = ''.join(x.lower() for x in message if x.isalpha())  # Ensure message is lowercase letters only
        encrypted_message = []
        for l in message:
            self.step_rotors()  # Step the rotors before each letter encryption
            encrypted_message.append(self.encrypt_letter(l))
        return ''.join(encrypted_message)

In [50]:
e = Enigma(key=[0, 0, 0], plugboard=["ab"])
enc = e.encrypt("the weather is clear")
e = Enigma(key=[0, 0, 0], plugboard=["ab"])
e.encrypt(enc)

'theweatherisclear'

In [46]:
a = {"a":"b"}
{v: k for k, v in a.items()}

{'b': 'a'}